In [ ]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
import PyPDF2
from transformers import pipeline
from google.colab import files

In [ ]:
# Load the question-answering pipeline
model_name = "distilbert-base-cased-distilled-squad"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# Function to extract text from a PDF file
def extract_text_from_pdf(file):
    text = ""
    reader = PyPDF2.PdfReader(file)
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()
    return text

In [ ]:
# Upload the PDF document
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving animal_2.pdf to animal_2.pdf


In [ ]:
# Extract text from the uploaded PDF document
with open(pdf_path, 'rb') as file:
    document_content = extract_text_from_pdf(file)

In [ ]:
document_content

"Lions are big cats that live in Africa. They have golden fur and a big, loud roar! Lions are known as the \nkings of the jungle because they are strong and brave.  \n \nElephants are the largest land animals on Earth. They have long trunks that they use to pick up food \nand water. Elephants are gentle giants and they love to take baths in the mud!  \n \nButterflies are beautiful insects with colorful wings. They start their life as caterpillars and then \ntransform into butterflies. Butterflies flutter from flower to flower, drinking sweet nectar.  \n \nDolphins are intelligent mammals that live in the ocean. They are known for their playful behavior \nand their ability to jump high out of the water. Dolphins communicate with each other by making \nclicking sounds.  \n \nPenguins are birds that cannot fly, but they are excellent swimmers. They live in cold places like \nAntarctica and have thick feathers to keep them warm. Penguins waddle on land and slide on their \nbellies on the i

In [ ]:
# Import the STOP_WORDS set from the spaCy library
from spacy.lang.en.stop_words import STOP_WORDS

# Define the context variable containing the text
context = document_content

# Tokenize the context and remove stopwords
tokens = context.split()
context_without_stopwords = [token for token in tokens if token.lower() not in STOP_WORDS]

# Join the remaining tokens back into a single string
context_without_stopwords_str = ' '.join(context_without_stopwords)

# Display the text after removing stopwords
print("Text after removing stopwords:")
print(context_without_stopwords_str)

In [ ]:
# Ask multiple questions in a loop

while True:
    # Ask a question
    question = input("Enter your question: ")

    # Check if the user wants to exit
    if question.lower() == 'exit':
        break

    # Prepare input for the question-answering pipeline
    QA_input = {
        'question': question,
        'context': document_content
    }

    # Perform question answering and obtain the result
    result = nlp(QA_input)

    # Print the predicted answer
    print("Answer:", result['answer'])

Enter your question: who lives in water
Answer: Crocodiles
Enter your question: who is tallest animal
Answer: Giraffes
Enter your question: who plays a lot
Answer: Elephants are gentle giants and they love to take baths in the mud!
Enter your question: what is sloth
Answer: sloths are excellent swimmers and climbers
Enter your question: is sloth an animal
Answer: sloths are excellent swimmers and climbers
Enter your question: what is fish
Answer: Seahorses
Enter your question: give names of carnivores animals
Answer: Cheetahs
Enter your question: how do they communicate
Answer: vocalizations
Enter your question: how do dolphines communicate?
Answer: vocalizations
Enter your question: how do dolphins communicate?
Answer: vocalizations
Enter your question: dolphins communicate through?
Answer: vocalizations
Enter your question: who is largest animal on earth?
Answer: Elephants
Enter your question: female peacocks are called?
Answer: peahens
Enter your question: what is other name of fema

In [ ]:
result

{'score': 0.0349874347448349,
 'start': 742,
 'end': 805,
 'answer': 'Dolphins communicate with each other by \nmaking clicking sounds'}

In [ ]:
import nltk
nltk.download('punkt')  # Download the Punkt tokenizer models if you haven't already

from nltk.tokenize import sent_tokenize

text = document_content

sentences = sent_tokenize(text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sentences

['Lions are big cats that live in Africa.',
 'They have golden fur and a big, loud roar!',
 'Lions are known as the \nkings of the jungle because they are strong and brave.',
 'Elephants are the largest land animals on Earth.',
 'They have long trunks that they use to pick up food \nand water.',
 'Elephants are gentle giants and they love to take baths in the mud!',
 'Butterflies are beautiful insects with colorful wings.',
 'They start their life as caterpillars and then \ntransform into butterflies.',
 'Butterflies flutter from flower to flower, drinking sweet nectar.',
 'Dolphins are intelligent mammals that live in the ocean.',
 'They are known for their playful behavior \nand their ability to jump high out of the water.',
 'Dolphins communicate with each other by making \nclicking sounds.',
 'Penguins are birds that cannot fly, but they are excellent swimmers.',
 'They live in cold places like \nAntarctica and have thick feathers to keep them warm.',
 'Penguins waddle on land and 

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans
import numpy as np

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
model = BertModel.from_pretrained('distilbert-base-cased-distilled-squad')

# Example text data
texts = sentences

# Tokenize input texts and generate BERT embeddings
encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    outputs = model(**encoded_texts)
    embeddings = outputs.last_hidden_state

# Flatten the embeddings
flattened_embeddings = embeddings.cpu().numpy().reshape(embeddings.shape[0], -1)

# Cluster the embeddings using K-means clustering without normalization
n_clusters = 3  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(flattened_embeddings)

# Print the cluster labels for each text
for i, text in enumerate(texts):
    print(f"Text: {text} - Cluster Label: {cluster_labels[i]}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at distilbert-base-cased-distilled-squad and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.se

Text: Lions are big cats that live in Africa. - Cluster Label: 1
Text: They have golden fur and a big, loud roar! - Cluster Label: 1
Text: Lions are known as the 
kings of the jungle because they are strong and brave. - Cluster Label: 1
Text: Elephants are the largest land animals on Earth. - Cluster Label: 1
Text: They have long trunks that they use to pick up food 
and water. - Cluster Label: 1
Text: Elephants are gentle giants and they love to take baths in the mud! - Cluster Label: 0
Text: Butterflies are beautiful insects with colorful wings. - Cluster Label: 0
Text: They start their life as caterpillars and then 
transform into butterflies. - Cluster Label: 1
Text: Butterflies flutter from flower to flower, drinking sweet nectar. - Cluster Label: 0
Text: Dolphins are intelligent mammals that live in the ocean. - Cluster Label: 1
Text: They are known for their playful behavior 
and their ability to jump high out of the water. - Cluster Label: 1
Text: Dolphins communicate with each

In [ ]:
cluster_labels

array([1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 0, 1, 1, 1,
       1, 1, 1, 1, 1], dtype=int32)

In [ ]:
len(text)

127

In [ ]:
len(cluster_labels)

71

In [ ]:
encoded_texts

{'input_ids': tensor([[  101,  7862,  1132,  ...,     0,     0,     0],
        [  101,  1220,  1138,  ...,     0,     0,     0],
        [  101,  7862,  1132,  ...,     0,     0,     0],
        ...,
        [  101,  1220, 25572,  ...,     0,     0,     0],
        [  101, 20164,  8223,  ...,     0,     0,     0],
        [  101,  1220,  1132,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Load pre-trained DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')

# Example data for text classification
texts = sentences
labels = cluster_labels  # Example labels (1: positive, 0: negative)

# Tokenize input texts
encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Split data into train and validation sets
#train_inputs, val_inputs, train_labels, val_labels = train_test_split(encoded_texts[0], labels, test_size=0.2, random_state=42)


encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
# Define the ratio for splitting the data
train_ratio = 0.8  # 80% for training, 20% for testing

# Calculate the index to split the data
split_index = int(len(labels) * train_ratio)

# Split data into train and validation sets
train_inputs, val_inputs = encoded_texts[:split_index], encoded_texts[split_index:]
train_labels, val_labels = labels[:split_index], labels[split_index:]

In [ ]:
# Create DataLoader for training and validation sets
train_dataset = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(train_labels))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

val_dataset = torch.utils.data.TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], torch.tensor(val_labels))
val_loader = DataLoader(val_dataset, batch_size=8)

# Load pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased-distilled-squad', num_labels=2)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Define loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
for epoch in range(3):  # Adjust number of epochs as needed
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids.long(), attention_mask=attention_mask.long(), labels=labels.long())
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Validation loop
model.eval()
val_preds = []
val_targets = []
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)

        val_preds.extend(preds.cpu().numpy())
        val_targets.extend(labels.cpu().numpy())

# Calculate metrics
val_accuracy = accuracy_score(val_targets, val_preds)
val_precision = precision_score(val_targets, val_preds, average='macro')  # Use 'macro' for multiclass
val_recall = recall_score(val_targets, val_preds, average='macro')  # Use 'macro' for multiclass
val_f1 = f1_score(val_targets, val_preds, average='macro')

# Print metrics
print(f'Epoch {epoch + 1} - Validation Accuracy: {val_accuracy:.4f}')
print(f'Epoch {epoch + 1} - Validation Precision: {val_precision:.4f}')
print(f'Epoch {epoch + 1} - Validation Recall: {val_recall:.4f}')
print(f'Epoch {epoch + 1} - Validation F1 Score: {val_f1:.4f}')

Epoch 3 - Validation Accuracy: 0.5333
Epoch 3 - Validation Precision: 0.4545
Epoch 3 - Validation Recall: 0.4667
Epoch 3 - Validation F1 Score: 0.3683


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
